In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.initializers import glorot_uniform

# Loading the Dataset

In [2]:
df = pd.read_csv('data/dataset.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(10)


,Handle,Party,Tweet - 1,Tweet - 2,Tweet - 3,Tweet - 4,Tweet - 5,Tweet - 6,Tweet - 7,Tweet - 8,...,Tweet - 191,Tweet - 192,Tweet - 193,Tweet - 194,Tweet - 195,Tweet - 196,Tweet - 197,Tweet - 198,Tweet - 199,Tweet - 200
0,RepDarrenSoto,Democrat,Gov. @ricardorossello's comments degrading wom...,Looks like Trump will end his discriminatory c...,For several years we sought to replace our sta...,Proud to announce that @fema awarded @PolkCoun...,STURDY Bill passed @EnergyCommerce Cmte today ...,Enough is enough! Billionaire super predator J...,We continue our efforts to provide American Ci...,We're committed to defending quality &amp; aff...,...,We've hosted several DREAMer interns in our of...,"As 2020 nears, Puerto Rico statehood looms lar...",Honored to vote for Violence Against Women Act...,Having knocked on thousands of doors in the di...,We are committed to giving all our kids a #Hea...,Today in @EnergyCommerce we began taking actio...,Proud to work to protect the #Everglades for a...,In @EnergyCommerce today voting on ambitious @...,My honor and you can count on our support! htt...,It is 2019 and women are still paid 80 cents f...
1,SenJackyRosen,Democrat,Officially unveiled my bipartisan #VeteransJob...,The @USWNT couldn’t have made our country prou...,My team was happy to attend the grand opening ...,Excited to introduce the bipartisan Veterans J...,We have work to do in Nevada to increase the n...,I put my career on hold to take care of my agi...,"Representation matters, and I’m proud to have ...",Productive discussion this morning with immigr...,...,"Thankful for organizations like @TahoeAgency, ...",Congratulations to @ClarkCountyNV for achievin...,"At this week’s @HSGAC hearing, I asked the nom...","In 2018, wildfires scorched over 230,000 acres...",7 years ago today we made a promise to protect...,I’m grateful for organizations such as these t...,Honored to take part in today’s @DreamBigNV ri...,#ClimateChange is real and we need #BoldClimat...,I’m co-sponsoring the International Climate Ac...,Honored to meet Nevada’s freshman class of cad...
2,RepAlLawsonJr,Democrat,Our troops are the best America has to offer –...,More than 130 million Americans live with a pr...,This is the longest period of time in history ...,The annual Capitol Fourth celebrations are a m...,Today is a chance for our nation to come toget...,Great morning speaking with the teens at Data ...,The deaths at the border are a tragedy that we...,Veteran suicide is a sustained national crisis...,...,Food stamps for 38 million low-income American...,The power of your vote! https://t.co/4ZFhSi4geT,Congrats to #FL05 #App Challenge winners #Tall...,New office for the #116thCongress. Swing by 14...,There is no one more deserving to be Speaker o...,Join me this morning for the ceremonial sweari...,Wishing you a very Merry Christmas! https://t....,The resolution is not the end of this critical...,It's unfortunate my Republican colleagues play...,I’m pleased that the House has worked to provi...
3,RepEspaillat,Democrat,You cannot defend such evil and @SecretaryAcos...,We must investigate the toxic culture at the @...,Congratulations US Women’s soccer team on thei...,Thank you #NYPD Hate Crimes Task Force for eff...,I’m down to go to the mat to get @asvpxrocky b...,I commend @NYGovCuomo and the New York State L...,"By refusing to release his taxes, #Trump has v...",@realDonaldTrump has repeatedly refused to com...,...,"Peaceful protestors are being murdered, arrest...",Sending @realDonaldTrump 73 “birthday” candles...,Sending @realDonaldTrump 73 “birthday” candles...,#Kellyanne Conway should absolutely be removed...,Miguel Angel lived to talk about the treatment...,"Today, I addressed the issue of U.S. arms sale...",Today marks the third anniversary of the tragi...,By attempting to keep immigrants from accessin...,Where is the American outcry over the inhumane...,Immigrants have always contributed to our nati...
4,kyrstensinema,Democrat,Part 1 of my first-ever birt

# Pre-processing

In [8]:
df['Party'] = pd.Categorical(df.Party)
df['Party'] = pd.get_dummies(df['Party'], drop_first=True)
# df[df['Party'] == 0]
# df.head()


<b>0</b> - Democrat <br>
<b>1</b> - Republican

In [9]:
X = df[df.columns[2:]]
Y = df['Party']
print(X.shape)
print(Y.shape)

(451, 200)
(451,)


In [10]:
print(X.iloc[0,:10])

Tweet - 1     Gov. @ricardorossello's comments degrading wom...
Tweet - 2     Looks like Trump will end his discriminatory c...
Tweet - 3     For several years we sought to replace our sta...
Tweet - 4     Proud to announce that @fema awarded @PolkCoun...
Tweet - 5     STURDY Bill passed @EnergyCommerce Cmte today ...
Tweet - 6     Enough is enough! Billionaire super predator J...
Tweet - 7     We continue our efforts to provide American Ci...
Tweet - 8     We're committed to defending quality &amp; aff...
Tweet - 9     Robocalls aren’t just annoying. Many are outri...
Tweet - 10    This is why we continue to fight @jediabetical...
Name: 0, dtype: object


### Removing special characters

In [11]:
for col in X.columns:
    X[col] = X[col].str.replace('&amp;', "and")
    X[col] = X[col].str.replace('\xa0', " ")
    X[col] = X[col].str.replace('\u2003', " ")


C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tshan\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [12]:
print(X.iloc[0,122])

We’re indeed in a constitutional crisis and must defend our democracy! AG Barr should be held in contempt for refusal to testify and submit docs. As co-equal branch, Congress has a constitutional obligation to oversight, combat corruption and get to the truth!
https://t.co/vjZ2P7PSWl


### Removing links

In [13]:
def remove_link(sent):
    sent = str(sent)
    return sent[:sent.find("https://")]

In [14]:
X = X.apply(np.vectorize(remove_link))

In [15]:
print(X.iloc[0,122])

We’re indeed in a constitutional crisis and must defend our democracy! AG Barr should be held in contempt for refusal to testify and submit docs. As co-equal branch, Congress has a constitutional obligation to oversight, combat corruption and get to the truth!



### Removing punctuations and converting sentences into lowercase

In [16]:
import re
from string import punctuation
punctuation = punctuation + "—"
regex = re.compile('[%s]' % re.escape(punctuation))

def remove_punctuations(sentence):
    return regex.sub('', sentence)
    

In [17]:
X = X.apply(np.vectorize(remove_punctuations))
X = X.apply(np.vectorize(str.lower))

In [18]:
print(X.iloc[0,122])

we’re indeed in a constitutional crisis and must defend our democracy ag barr should be held in contempt for refusal to testify and submit docs as coequal branch congress has a constitutional obligation to oversight combat corruption and get to the truth



In [19]:
X.loc[63, 'Tweet - 16']

'the 3rd of july means one thing  thunder on the canal at alderbrook i was a guest timer for the oyster eating contest the rapid consumption of mass quantities of shellfish was in a word disgusting  somehow i’m not sure this is what our founding fathers envisioned 😉 '

# Modelling
<b><hr>

## Splitting into training and testing sets


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=420)

## Using Tokenizer and creating a vocabulary

In [21]:
t = Tokenizer(oov_token="UNK")
a = []

In [22]:
for i in X_train.index:
    a.append(X_train.loc[i].tolist())
    
final_docs = [j for i in a for j in i]

In [23]:
print("Len of a: {}".format(len(a)))
print("Len of a: {}".format(len(final_docs)))

Len of a: 360
Len of a: 72000


### Creating a vocabulary

In [24]:
t.fit_on_texts(final_docs)

In [25]:
vocab_size = len(t.word_index) + 1
print("Vocabulary size: {}".format(vocab_size))

Vocabulary size: 71746


In [26]:
def get_max_length(X_train):
    max_len = 0
    for col in X_train.columns:
        temp = len(max(X_train[col], key=len).split())

        if temp > max_len:
            max_len = temp
            tweet = max(X_train[col], key=len)
            column = col

    return max_len

In [27]:
max_sent_len = get_max_length(X_train) + 1
print("Maximum sentence length: {}".format(max_sent_len))

Maximum sentence length: 74


### Encoding the sentences

In [28]:
def convert_sentences(sentence, word_to_index):
    encoded = np.zeros((1, max_sent_len))
    sentence_words = sentence.lower().split()
    j = 0
    for w in sentence_words:
        encoded[0, j] = word_to_index[w]
        j += 1
    
    return encoded

In [29]:
encoded_X_train = X_train.copy()

In [30]:
for col in encoded_X_train.columns:
    encoded_X_train[col] = X_train[col].apply(convert_sentences, args=(t.word_index,))

In [31]:
encoded_X_train.iloc[0,0]

array([[1.4701e+04, 1.1358e+04, 2.7000e+02, 3.5300e+03, 1.5500e+02,
        6.8380e+03, 7.4000e+01, 1.4700e+02, 2.1200e+02, 5.1600e+02,
        3.1500e+02, 5.8100e+02, 6.0660e+03, 4.0270e+03, 4.0000e+00,
        1.6300e+02, 4.7490e+03, 1.9100e+02, 6.8390e+03, 3.9050e+03,
        1.1980e+03, 5.0000e+00, 5.1700e+02, 5.0000e+00, 8.1100e+02,
        8.6600e+02, 3.3219e+04, 3.0080e+03, 4.0000e+01, 3.3900e+02,
        2.8200e+02, 2.1600e+02, 2.0000e+00, 6.7700e+02, 5.7000e+01,
        9.8500e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

## Embedding Layer

In [32]:
emb_dim = 50

In [34]:
embedding_layer = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_sent_len, trainable=False)
trained_embedding_layer = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_sent_len, trainable=True)

In [37]:
from tensorflow.keras.layers import Reshape

In [40]:
m = Reshape(target_shape=(90200, 1))(X)

ValueError: Cannot reshape a tensor with 90200 elements to shape [451,90200,1] (40680200 elements) for 'reshape_2/Reshape' (op: 'Reshape') with input shapes: [451,200], [3] and with input tensors computed as partial shapes: input[1] = [451,90200,1].

In [ ]:
m